In [1]:
import os
import pathlib
from collections import deque
from itertools import combinations

import geopy as gpd
import pandas as pd

import numpy as np

import folium
import openrouteservice as ors
from shapely.geometry import Point, LineString

import warnings
warnings.filterwarnings('ignore')

project_path = pathlib.Path.cwd().parent.parent

with open(str(project_path) + "/api/openrouteservice.pem") as f_in:
    ORS_API = f_in.read()

PROFILE = "driving-car"
ORS_OPTIONS = {"avoid_borders": "all"}

MAX_PER_MIN = 30
WINDOW_SEC = 60.0
EPS = 0.02
BASE_SLEEP = 0.20

MAX_PER_SEC = 1
SEC_WINDOW = 1.0

MAX_RETRIES = 3
WRITE_POINTS = True

SNAP_RADIUS_M = 10000
USE_RADIUSES = [-1, -1]

In [29]:
client = ors.Client(key=ORS_API)
town_halls_coor = pd.read_excel(str(project_path) + "/data/Coordinates/coor.xlsx")
town_halls_coor['coordinates'] = town_halls_coor.apply(lambda row: (row['lon'], row['lat']), axis=1)
town_halls_coor

,name,lat,lon,coordinates
0,กรุงเทพมหานคร,13.753378,100.501941,"(100.5019413, 13.7533776)"
1,กระบี่,8.059001,98.917394,"(98.9173938, 8.0590009)"
2,กาญจนบุรี,14.004018,99.548992,"(99.5489918, 14.0040177)"
3,กาฬสินธุ์,16.440808,103.538121,"(103.5381206, 16.4408083)"
4,กำแพงเพชร,16.555072,99.511041,"(99.5110407, 16.5550717)"
...,...,...,...,...
72,อำนาจเจริญ,15.901520,104.622743,"(104.6227431, 15.9015203)"
73,อุดรธานี,17.415249,102.787417,"(102.7874174, 17.4152487)"
74,อุตรดิตถ์,17.629695,100.095969,"(100.0959692, 17.6296952)"
75,อุทัยธานี,15.373556,100.038913,"(100.0389132, 15.373556)"


In [30]:
coords = tuple(town_halls_coor['coordinates'])
coords

((100.5019413, 13.7533776),
 (98.9173938, 8.0590009),
 (99.5489918, 14.0040177),
 (103.5381206, 16.4408083),
 (99.5110407, 16.5550717),
 (102.8359805, 16.4423433),
 (102.1027886, 12.6103945),
 (101.0701405, 13.6876388),
 (100.9833311, 13.3618997),
 (100.1249575, 15.1853598),
 (102.0329667, 15.8080698),
 (99.1925874, 10.5215575),
 (99.8142679, 19.920675),
 (98.9708332, 18.8371264),
 (99.611531, 7.5578226),
 (102.5181214, 12.2433484),
 (99.1254836, 16.8844362),
 (101.2013854, 14.2120628),
 (100.0586394, 13.7667973),
 (104.7823595, 17.4079822),
 (102.1000962, 14.9723581),
 (99.9634825, 8.4185082),
 (100.1138136, 15.69076),
 (100.5144365, 13.8625111),
 (101.7979613, 6.190088),
 (100.733478, 18.7965004),
 (103.6850562, 18.3342414),
 (103.1065755, 14.9441768),
 (100.5246592, 14.0213121),
 (99.7979013, 11.8123177),
 (101.3296187, 14.1097008),
 (101.2501995, 6.867817),
 (100.6112162, 14.3338975),
 (99.879867, 19.1914635),
 (98.5294709, 8.4260174),
 (100.072822, 7.6169441),
 (100.3619731, 16.41

In [31]:
pairs_coords = tuple(combinations(coords, 2))

town_halls_pairs_OD = []
for key, value in enumerate(pairs_coords):
    dummy_node = {"origin": value[0], "destination": value[1]}
    town_halls_pairs_OD.append(dummy_node)

town_halls_pairs_OD

[{'origin': (100.5019413, 13.7533776), 'destination': (98.9173938, 8.0590009)},
 {'origin': (100.5019413, 13.7533776),
  'destination': (99.5489918, 14.0040177)},
 {'origin': (100.5019413, 13.7533776),
  'destination': (103.5381206, 16.4408083)},
 {'origin': (100.5019413, 13.7533776),
  'destination': (99.5110407, 16.5550717)},
 {'origin': (100.5019413, 13.7533776),
  'destination': (102.8359805, 16.4423433)},
 {'origin': (100.5019413, 13.7533776),
  'destination': (102.1027886, 12.6103945)},
 {'origin': (100.5019413, 13.7533776),
  'destination': (101.0701405, 13.6876388)},
 {'origin': (100.5019413, 13.7533776),
  'destination': (100.9833311, 13.3618997)},
 {'origin': (100.5019413, 13.7533776),
  'destination': (100.1249575, 15.1853598)},
 {'origin': (100.5019413, 13.7533776),
  'destination': (102.0329667, 15.8080698)},
 {'origin': (100.5019413, 13.7533776),
  'destination': (99.1925874, 10.5215575)},
 {'origin': (100.5019413, 13.7533776), 'destination': (99.8142679, 19.920675)},
 {'

In [ ]:
result = {}
for k, item in enumerate(town_halls_pairs_OD, start=1):
    prov_o, prov_d = item['origin'], item['destiantion']
    
    route = client.directions(
    coordinates = [item['origin'], item['destiantion']], 
    profile = 'driving-car',
    optimize_waypoints = True,
    format = 'geojson',
    instructions = False,
    validate = False,
    radiuses = USE_RADIUSES)

OD pair 1: {'origin': (100.5019413, 13.7533776), 'destination': (98.9173938, 8.0590009)}
OD pair 2: {'origin': (100.5019413, 13.7533776), 'destination': (99.5489918, 14.0040177)}
OD pair 3: {'origin': (100.5019413, 13.7533776), 'destination': (103.5381206, 16.4408083)}
OD pair 4: {'origin': (100.5019413, 13.7533776), 'destination': (99.5110407, 16.5550717)}
OD pair 5: {'origin': (100.5019413, 13.7533776), 'destination': (102.8359805, 16.4423433)}
OD pair 6: {'origin': (100.5019413, 13.7533776), 'destination': (102.1027886, 12.6103945)}
OD pair 7: {'origin': (100.5019413, 13.7533776), 'destination': (101.0701405, 13.6876388)}
OD pair 8: {'origin': (100.5019413, 13.7533776), 'destination': (100.9833311, 13.3618997)}
OD pair 9: {'origin': (100.5019413, 13.7533776), 'destination': (100.1249575, 15.1853598)}
OD pair 10: {'origin': (100.5019413, 13.7533776), 'destination': (102.0329667, 15.8080698)}
OD pair 11: {'origin': (100.5019413, 13.7533776), 'destination': (99.1925874, 10.5215575)}
OD

In [33]:
test_coord = pairs_coords[0]
test_coord

((100.5019413, 13.7533776), (98.9173938, 8.0590009))

In [36]:
client = ors.Client(key=ORS_API)
route = client.directions(
    coordinates=test_coord, 
    profile='driving-car',
    optimize_waypoints=True,
    format='geojson',
    instructions=False,
    validate=False,
    radiuses=USE_RADIUSES)

print(route)

{'type': 'FeatureCollection', 'bbox': [98.783732, 8.056026, 100.502202, 13.753334], 'features': [{'bbox': [98.783732, 8.056026, 100.502202, 13.753334], 'type': 'Feature', 'properties': {'way_points': [0, 5095], 'summary': {'distance': 777957.8, 'duration': 33763.1}}, 'geometry': {'coordinates': [[100.502202, 13.753334], [100.502188, 13.753257], [100.502104, 13.752637], [100.502055, 13.752273], [100.501994, 13.751925], [100.501946, 13.75171], [100.501822, 13.750441], [100.501785, 13.7501], [100.501696, 13.749332], [100.501586, 13.74825], [100.501525, 13.748152], [100.501332, 13.748045], [100.50125, 13.747946], [100.501187, 13.747595], [100.501089, 13.747057], [100.500997, 13.746665], [100.500874, 13.746167], [100.50081, 13.745905], [100.500651, 13.745345], [100.50054, 13.744857], [100.500239, 13.744944], [100.499345, 13.745147], [100.499215, 13.744535], [100.499207, 13.744499], [100.499144, 13.7442], [100.499074, 13.743873], [100.498972, 13.743423], [100.498834, 13.742939], [100.498716,

In [41]:
route['features'][0].get("properties")

{'way_points': [0, 5095],
 'summary': {'distance': 777957.8, 'duration': 33763.1}}

In [ ]:
m = folium.Map(
    location=list(reversed([100.5019413, 13.7533776])),
    tiles='Cartodb Positron',
    zoom_start=13
)

test_coords = [[100.5019413, 13.7533776], [98.9173938, 8.0590009]]

client = ors.Client(key=ORS_API)
test_route = client.directions(
    coordinates=test_coords, 
    profile='driving-car',
    optimize_waypoints=True,
    format='geojson',
    instructions=False,
    validate=False,
    radiuses=USE_RADIUSES)

folium.PolyLine(locations=[list(reversed(coord)) for coord in test_route['features'][0]['geometry']['coordinates']], color='blue').add_to(m)

m

In [ ]:
num_coords = len(coords)
batch = num_coords // 7
count = 0

while count < num_coords:
    sub_coords = coords[count: count + batch]
    routes = client.directions(coordinates=coords, 
                               profile='driving-car',
                               optimize_waypoints=True,
                               format='geojson')
    print(routes)
    
    count += batch